In [88]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

img_height = 300
img_width = 300

K.clear_session() 

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=2,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], 
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.5,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

#학습된 weight의 경로를 지정
weights_path = 'C:\\Users\\user\\keras\\ssd_keras\\ssd300_pascal_07+12_epoch-08_loss-1.9471_val_loss-1.9156.h5'

model.load_weights(weights_path, by_name=True)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [96]:
#영상의 경로를 지정하고 프레임 캡쳐
#나중에는 비디오 캡쳐를 함과 동시에 input_images리스트에 곧바로 넣어버려서, 불필요한 이미지 입출력 과정을 줄이자
#1초에 4프레임 캡쳐로 바꿈 (6프레임마다 저장)
vidcap = cv2.VideoCapture('C:\\Users\\user\Desktop\\video5.mp4')
success,imagefile = vidcap.read()
count = 0
while success:
    if(count%6==0):
#프레임 캡쳐를 저장할 경로
        cv2.imwrite("C:\\Users\\user\Desktop\\frames\\frame%d.jpg" % count, imagefile)    
    success,imagefile = vidcap.read()
    count += 1

orig_images = [] 
input_images = [] 

# range는 추후 영상 플레이 타임 정보를 사용할 수 있도록 변경
for i in range(0,320):
    if(i%6==0):
#프레임 캡쳐를 불러오는 경로
        img_path = 'C:\\Users\\user\Desktop\\frames\\frame%d.jpg'%i
        #print(img_path)
        orig_images.append(imread(img_path))
        img = image.load_img(img_path, target_size=(img_height, img_width))
        img = image.img_to_array(img)
        img = np.array(img)
        input_images.append(img)
        
input_images = np.array(input_images)
orig_images = np.array(orig_images)

num_of_frames = 16
counting = 0
saving_bounding_boxes = []

print("Predicted boxes:\n")
print('   class   conf xmin   ymin   xmax   ymax')

# range는 추후 변수를 사용할 수 있도록 변경
for i in range(0, 4):
    y_pred = model.predict(input_images[i*num_of_frames:i*num_of_frames + num_of_frames])
    confidence_threshold = 0.4

    y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
    np.set_printoptions(precision=2, suppress=True, linewidth=90)

    for j in range(0, num_of_frames):
        print('frame :',counting)
#        print(y_pred_thresh[j])
        for box in y_pred_thresh[j]:
            # Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
            xmin = box[2] * orig_images[0].shape[1] / img_width
            ymin = box[3] * orig_images[0].shape[0] / img_height
            xmax = box[4] * orig_images[0].shape[1] / img_width
            ymax = box[5] * orig_images[0].shape[0] / img_height
            print('xmin : ',xmin, '  ymin : ',ymin, '  xmax : ',xmax, '  ymax : ',ymax)
            # 균열이 탐지된 프레임과 b-box 정보가 saving_bounding_boxes <- 여기에 저장됨
            saving_bounding_boxes.append([counting, xmin,ymin,xmax,ymax])
        counting += 6
        if(counting>320): break;
            
'''
5. 방향 설정까지 했다면, 그 방향에서 edge가 몇 픽셀 떨어져 있는지를 재고, 각 skeleton line상의 pixel에 그 정보를 저장하자!
6. 거기서 max값을 return하자
'''

Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
frame : 0
xmin :  27.872576166788736   ymin :  -3.3622314453125   xmax :  1303.6754740397134   ymax :  107.8629150390625
frame : 6
xmin :  26.658366133371988   ymin :  -3.2167266845703124   xmax :  1307.7108833821615   ymax :  108.52862548828125
frame : 12
xmin :  23.838637828826904   ymin :  -3.223650360107422   xmax :  1305.6300118001302   ymax :  108.962451171875
frame : 18
xmin :  28.83223929087321   ymin :  -2.605290412902832   xmax :  1306.2607381184896   ymax :  108.52705078125
frame : 24
xmin :  22.28019380569458   ymin :  -2.4522186279296876   xmax :  1308.9802823893228   ymax :  111.5382568359375
frame : 30
xmin :  26.713252067565918   ymin :  -3.5178924560546876   xmax :  1310.0238159179687   ymax :  111.73031005859374
frame : 36
xmin :  24.485044339497883   ymin :  -3.776943588256836   xmax :  1302.6157784016928   ymax :  110.70706787109376
frame : 42
xmin :  27.910456110636392   ymin :  -4.398983764648437   xmax :

'\n5. 방향 설정까지 했다면, 그 방향에서 edge가 몇 픽셀 떨어져 있는지를 재고, 각 skeleton line상의 pixel에 그 정보를 저장하자!\n6. 거기서 max값을 return하자\n'

In [97]:
from skimage import io

cropped_frames = []

for i in range(0, len(saving_bounding_boxes)):
    frame_count = saving_bounding_boxes[i][0]//6
    frame = orig_images[frame_count]
    if(saving_bounding_boxes[i][1] < 0):
        saving_bounding_boxes[i][1] = 0
    xmin = int(saving_bounding_boxes[i][1])
    if(saving_bounding_boxes[i][2] < 0):
        saving_bounding_boxes[i][2] = 0
    ymin = int(saving_bounding_boxes[i][2])
    xmax = int(saving_bounding_boxes[i][3])
    ymax = int(saving_bounding_boxes[i][4])
    print(xmin,ymin,xmax,ymax)
    cropped_frame = orig_images[frame_count][ymin:ymax, xmin:xmax, :]
    cropped_frame = cropped_frame.astype('uint8')
    img_path = '../../Desktop/test/%d.jpg'%frame_count
    print(img_path)
    cropped_frames.append(cropped_frame)
    io.imsave(img_path, cropped_frame)

27 0 1303 107
../../Desktop/test/0.jpg
26 0 1307 108
../../Desktop/test/1.jpg
23 0 1305 108
../../Desktop/test/2.jpg
28 0 1306 108
../../Desktop/test/3.jpg
22 0 1308 111
../../Desktop/test/4.jpg
26 0 1310 111
../../Desktop/test/5.jpg
24 0 1302 110
../../Desktop/test/6.jpg
27 0 1302 108
../../Desktop/test/7.jpg
26 0 1301 107
../../Desktop/test/8.jpg
23 0 1304 112
../../Desktop/test/9.jpg
20 0 1300 110
../../Desktop/test/10.jpg
22 0 1299 108
../../Desktop/test/11.jpg
21 0 1301 109
../../Desktop/test/12.jpg
23 0 1303 110
../../Desktop/test/13.jpg
20 0 1295 106
../../Desktop/test/14.jpg
8 0 1295 111
../../Desktop/test/15.jpg
16 0 1306 112
../../Desktop/test/16.jpg
13 0 1331 120
../../Desktop/test/18.jpg
0 2 1307 127
../../Desktop/test/34.jpg


In [98]:
# 1. Image binarization(Sauvola's method) using Pw and Pl, respectively
# 오래 걸리는 문제가 있음

import time
import matplotlib
import matplotlib.pyplot as plt
import cv2
from skimage import io
from skimage import data
from skimage.color import rgb2gray
from skimage.data import page
from skimage.filters import (threshold_sauvola)
from PIL import Image

start_time = time.time() 
matplotlib.rcParams['font.size'] = 9

sauvola_frames_Pw_bw = []
sauvola_frames_Pl_bw = []
sauvola_frames_Pw = []
sauvola_frames_Pl = []

# Upload the image
for i in range(0,len(cropped_frames)):
    img = cropped_frames[i]
    img_gray = rgb2gray(img)

    # 논문에선 각각 70,180이었으나 여기선 홀수 input만 가능
    window_size_Pw = 71
    window_size_Pl = 181
    thresh_sauvola_Pw = threshold_sauvola(img_gray, window_size=window_size_Pw, k=0.42)
    thresh_sauvola_Pl = threshold_sauvola(img_gray, window_size=window_size_Pl, k=0.18)

    #Below are the converted images through Sauvola's method.
    # _bw will contain 0 or 1, not true or false. bw means black or white.
    binary_sauvola_Pw = img_gray > thresh_sauvola_Pw
    binary_sauvola_Pl = img_gray > thresh_sauvola_Pl
    binary_sauvola_Pw_bw = img_gray > thresh_sauvola_Pw
    binary_sauvola_Pl_bw = img_gray > thresh_sauvola_Pl

    binary_sauvola_Pw_bw.dtype = 'uint8'
    binary_sauvola_Pl_bw.dtype = 'uint8'

    binary_sauvola_Pw_bw *= 255
    binary_sauvola_Pl_bw *= 255
    
    sauvola_frames_Pw_bw.append(binary_sauvola_Pw_bw)
    sauvola_frames_Pl_bw.append(binary_sauvola_Pl_bw)
    sauvola_frames_Pw.append(binary_sauvola_Pw)
    sauvola_frames_Pl.append(binary_sauvola_Pl) 
    
    img_path_Pw = '../../Desktop/Sauvola/Sauvola_Pw_%d.jpg'%i
    img_path_Pl = '../../Desktop/Sauvola/Sauvola_Pl_%d.jpg'%i
    
    io.imsave(img_path_Pw, binary_sauvola_Pw_bw)
    io.imsave(img_path_Pl, binary_sauvola_Pl_bw)

print("start_time", start_time)
print("--- %s seconds ---" %(time.time() - start_time))

c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\util\arraycrop.py:177: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropped = ar[slices]


start_time 1538145722.4002447
--- 102.0319037437439 seconds ---


In [99]:
# 2. Extract the skeletons of each images

from skimage.morphology import skeletonize
from skimage.util import invert

skeleton_frames_Pw = []
skeleton_frames_Pl = []

for i in range(0,len(cropped_frames)):
# Invert the binarized images
    img_Pw = invert(sauvola_frames_Pw[i])
    img_Pl = invert(sauvola_frames_Pl[i])

    # Below are skeletonized images
    skeleton_Pw = skeletonize(img_Pw)
    skeleton_Pl = skeletonize(img_Pl)

    # Convert true/false to 1/0 to save it as image
    skeleton_Pw.dtype = 'uint8'
    skeleton_Pl.dtype = 'uint8'

    skeleton_Pw *= 255
    skeleton_Pl *= 255

    skeleton_frames_Pw.append(skeleton_Pw)
    skeleton_frames_Pl.append(skeleton_Pl)   
    
    img_path_Pw = "../../Desktop/Skeleton/skeleton_Pw_%d.jpg"%i
    img_path_Pl = "../../Desktop/Skeleton/skeleton_Pl_%d.jpg"%i
    io.imsave(img_path_Pw, skeleton_Pw)
    io.imsave(img_path_Pl, skeleton_Pl)
    

c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pw_1.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pw_3.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pw_4.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pw_5.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pw_15.jpg is a low contrast image
  wa

In [100]:
# 3. Detect the edges of each images
### edge detection 할 때, 좋은 parameter를 찾아야 한다. 지금은 edge가 너무 두꺼움 (overestimation됨) ###
import numpy as np
from scipy import ndimage as ndi
from skimage import feature

edges_frames_Pw = []
edges_frames_Pl = []

for i in range(0,len(cropped_frames)):
    # Compute the Canny filter for two values of sigma
    # canny(image, sigma=1.0, low_threshold=None, high_threshold=None, mask=None, use_quantiles=False)
    # sigma가 1이었으나, 0.1로 조정하여 실제 균열 edge와 거의 같게 만듦.
    # 정확도에서 문제가 생긴다면 1. skeleton의 방향 설정 방법을 바꾸던가, 2. 여기서 시그마 값을 살짝 늘리거나 줄여가면서 정확도를 테스트 해볼 것
    edges_Pw = feature.canny(sauvola_frames_Pw[i], 0.09)
    edges_Pl = feature.canny(sauvola_frames_Pl[i], 0.09)

    edges_Pw.dtype = 'uint8'
    edges_Pl.dtype = 'uint8'

    edges_Pw *= 255
    edges_Pl *= 255

    edges_frames_Pw.append(edges_Pw)
    edges_frames_Pl.append(edges_Pl)
    
    img_path_Pw = "../../Desktop/edges/edges_Pw_%d.jpg"%i
    img_path_Pl = "../../Desktop/edges/edges_Pl_%d.jpg"%i
    
    io.imsave(img_path_Pw, edges_Pw)
    io.imsave(img_path_Pl, edges_Pl)


In [101]:
'''
np.set_printoptions(threshold=np.inf)
print(skeleton_frames_Pw[0].shape)

crop_skeleton = np.zeros((120,146))
crop_edges = np.zeros((120,146))

for i in range(0,120):
    for j in range(0,146):
        if(skeleton_frames_Pw[0][i][j] == 255): crop_skeleton[i][j] = crop_edges[i][j] = 1
        else: crop_skeleton[i][j] = crop_edges[i][j] = 0
        
        
np.savetxt("../../Desktop/skeleton.txt", crop_skeleton, '%d')
np.savetxt("../../Desktop/edge.txt", crop_edges, '%d')



count = 0
for i in range(skeleton_frames_Pw[0].shape[0]):
    for j in range(skeleton_frames_Pw[0].shape[1]):
        if(skeleton_frames_Pw[0][i][j] == 255): count+=1
            
print(count)
'''

#edges_frames_Pw[0]

(107, 1276)


IndexError: index 107 is out of bounds for axis 0 with size 107

In [ ]:
#Crack만이 detection되어서 넘어왔다는 가정이 있어야 함. 아니면 외부 배경 이미지도 균열 계산에 포함 됨

import queue
import math

#5픽셀이 기준 or above
dx_dir_right = [-5,-5,-5,-4,-3,-2,-1,0,1,2,3,4,5,5]
dy_dir_right = [0,1,2,3,4,5,5,5,5,5,4,3,2,1]

dx_dir_left = [5,5,5,4,3,2,1,0,-1,-2,-3,-4,-5,-5]
dy_dir_left = [0,-1,-2,-3,-4,-5,-5,-5,-5,-5,-4,-3,-2,-1]

dx_bfs = [-1,-1,0,1,1,1,0,-1]
dy_bfs = [0,1,1,1,0,-1,-1,-1]

start_time = time.time() 

for k in range(0,len(skeleton_frames_Pw)):
    print('--------------',k,'-----------------')
    start = [0,0]
    next = []
    q = queue.Queue()
    q.put(start)

    len_x = skeleton_frames_Pw[k].shape[0]
    len_y = skeleton_frames_Pw[k].shape[1]

    visit = np.zeros((len_x,len_y))
    count = 0
    crack_width_list = []

    while(q.empty() == 0):
        next = q.get()
        x = next[0]
        y = next[1]
        right_x = right_y = left_x = left_y = -1


        if(skeleton_frames_Pw[k][x][y] == 255):
            for i in range(0, len(dx_dir_right)):
                right_x = x + dx_dir_right[i]
                right_y = y + dy_dir_right[i]
                if(right_x<0 or right_y<0 or right_x>=len_x or right_y>=len_y): 
                    right_x = right_y = -1
                    continue;
                if(skeleton_frames_Pw[k][right_x][right_y] == 255): break;
                if(i==13): right_x = right_y = -1

            if(right_x == -1): 
                right_x = x
                right_y = y

            for i in range(0, len(dx_dir_left)):
                left_x = x + dx_dir_left[i]
                left_y = y + dy_dir_left[i]
                if(left_x <0 or left_y<0 or left_x >=len_x or left_y>=len_y): 
                    left_x = left_y = -1
                    continue;
                if(skeleton_frames_Pw[k][left_x][left_y] == 255): break;
                if(i==13): left_x = left_y = -1

            if(left_x == -1): 
                left_x = x
                left_y = y

            base = right_y - left_y
            height = right_x - left_x
            hypotenuse = math.sqrt(base*base + height*height)

            if(base==0 and height != 0): theta = 90.0
            elif(base==0 and height == 0): continue
            else: theta = math.degrees(math.acos((base * base + hypotenuse * hypotenuse - height * height)/(2.0 * base * hypotenuse)))



            theta += 90
            dist = 0

            for i in range(0,2):


                if(theta>360): theta -= 360
                elif(theta<0): theta += 360    
                #print(theta)
                #print('x : ',x,'y : ',y)
                pix_x=x
                pix_y=y

                #theta에러는 나중에 고치고
                #모든 프레임을 처리하기 위한 루프 먼저 돌릴것
                #결과 값을 UI와 연동하자

                if(theta>0 and theta<90):
                    ratio = abs(math.tan(theta))
                    while(1):
                        if((x - pix_x + 1)/(pix_y - y + 1)> ratio): pix_y+=1
                        else: pix_x-=1

                        if(pix_x<0 or pix_y<0 or pix_x>=len_x or pix_y>=len_y):
                            pix_x = x
                            pix_y = y
                            break;
                        if(edges_frames_Pw[k][pix_x][pix_y]==255): break;

                elif(theta>90 and theta<180):
                    ratio = abs(math.tan(theta))
                    while(1):
                        if((x - pix_x + 1)/(y - pix_y + 1)> ratio): pix_y-=1
                        else: pix_x-=1

                        if(pix_x<0 or pix_y<0 or pix_x>=len_x or pix_y>=len_y):
                            pix_x = x
                            pix_y = y
                            break;
                        if(edges_frames_Pw[k][pix_x][pix_y]==255): break;

                elif(theta>180 and theta<270):
                    ratio = abs(math.tan(theta))
                    while(1):
                        if((pix_x - x + 1)/(y - pix_y+ 1)> ratio): pix_y-=1
                        else: pix_x+=1

                        if(pix_x<0 or pix_y<0 or pix_x>=len_x or pix_y>=len_y):
                            pix_x = x
                            pix_y = y
                            break;
                        if(edges_frames_Pw[k][pix_x][pix_y]==255): break;     

                elif(theta>270 and theta<360):
                    ratio = abs(math.tan(theta))
                    while(1):
                        if((pix_x - x + 1)/(pix_y - y + 1)> ratio): pix_y+=1
                        else: pix_x+=1

                        if(pix_x<0 or pix_y<0 or pix_x>=len_x or pix_y>=len_y):
                            pix_x = x
                            pix_y = y
                            break;
                        if(edges_frames_Pw[k][pix_x][pix_y]==255): break;

                elif(theta == 0.0 or 360.0):
                     while(1):
                        pix_y+=1
                        if(pix_x<0 or pix_y<0 or pix_x>=len_x or pix_y>=len_y):
                            pix_x = x
                            pix_y = y
                            break;
                        if(edges_frames_Pw[k][pix_x][pix_y]==255): break;

                elif(theta == 90.0):
                    while(1):
                        pix_x-=1
                        if(pix_x<0 or pix_y<0 or pix_x>=len_x or pix_y>=len_y):
                            pix_x = x
                            pix_y = y
                            break;
                        if(edges_frames_Pw[k][pix_x][pix_y]==255): break;

                elif(theta == 180.0):
                    while(1):
                        pix_y-=1
                        if(pix_x<0 or pix_y<0 or pix_x>=len_x or pix_y>=len_y):
                            pix_x = x
                            pix_y = y
                            break;
                        if(edges_frames_Pw[k][pix_x][pix_y]==255): break;

                elif(theta == 270.0):
                     while(1):
                        pix_x+=1
                        if(pix_x<0 or pix_y<0 or pix_x>=len_x or pix_y>=len_y):
                            pix_x = x
                            pix_y = y
                            break;
                        if(edges_frames_Pw[k][pix_x][pix_y]==255): break;            

                dist += math.sqrt((y-pix_y)**2 + (x-pix_x)**2)
                theta += 180        

                #print('pix_x : ',pix_x,'pix_y : ',pix_y,'dist : ', dist,'\n')

            crack_width_list.append(dist)
        
            #해당 위치와 균열 폭을 힘께 저장하는 새로운 리스트 사용하기
        for i in range(0,8):
            next_x = x + dx_bfs[i]
            next_y = y + dy_bfs[i]

            if(next_x<0 or next_y<0 or next_x>=len_x or next_y>=len_y): continue;
            if(visit[next_x][next_y] == 0): 
                q.put([next_x,next_y])
                visit[next_x][next_y] = 1
        
    print(max(crack_width_list))
        
print("start_time", start_time)
print("--- %s seconds ---" %(time.time() - start_time))